## Problem statement

At first glance, training a variational autoencoder (VAE) seems pretty straightfoward -- it simply involves evaluating the evidence lower bound (ELBO) of the marginal data log likelihood and maximizing this ELBO by performing gradient descent on the parameters of the inference and generative networks, which are denoted by $\phi$ and $\theta$, respectively. However, it turns out that that in practice, a VAE trained using this standard procedure is prone to get stuck in a degenerate state. In this local optimum, the approximate posterior outputted by the inference network $q_{\phi}(\mathbf{z}|\mathbf{x})$ fails to capture any informative features contained in the input $x$ and instead regresses to the prior distribution $p(z)$ of the latent variable $z$. Because the latent variable produced by the encoder is meaningless, the decoder learns to ignore the latent variable entirely. Finally, the true model posterior $p_{\theta}(z|x) = \frac{p_{\theta}(x|z)p(z)}{p_{\theta}(x)}$ aligns with the approximate posterior by reverting back to the prior distribution $p(z)$ as well. This undesirable state is called posterior collapse and results in a useless model that hasn't learned any meaningful latent representation of the training data. 

Based on empirical results, the risk of posterior collapse appears to be highest for VAE models where the generator is parametrized by a strong autoregressive neural network such as an LSTM or PixelCNN, on when the data is discrete, such as when dealing with text.

The goal of the paper is simply to figure out how to avoid encountering posterior collapse during training. 

## Importance of problem

The utility of a VAE is measured by the extent to which the model is able to learn a useful latent representation of the input data, whether it be images, text or a high-dimensional set of features. Because posterior collapse results in an inference network that ignores $x$ and a generative network than ignores $z$, it poses a threat to this goal. Importantly, posterior collapse is not something that can be easily be averted, such as by reinitializing the weights or slightly tweaking the VAE architecture. Instead, it must be addressed directly. 

## Technical content (high level)

The contribution of the paper is motivated entirely by intuition about what happens during the early stages of VAE training. The following form of the ELBO is the most illuminating: 

$$ L(\mathbf{x}, \theta, \phi) = log p_{\theta}(\mathbf{x}) - D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x}) || p_{\theta}(\mathbf{z}|\mathbf{x}))$$ 

In this formulation of the ELBO, the first term represents the marginal data log likelihood and encourages the decoder to accurately reconstruct the input data, while the second term represents the KL divergence between the true model posterior and the approximate model posterior and encourages these two distributions to be alike.

The overall goal of training the VAE is to learn a latent representation $z_n$ corresponding to each $x_n$ that captures the useful information in $x_n$. The marginal data log likelihood term of the ELBO pushes the VAE to learn this meaningful latent representation -- if $z_n$ doesn't convey much useful information about $x_n$, then the best the decoder can do is to produce a crude distribution that is representative of an average value of $x$ in the training set. Such an output distribution would result in a relatively low observed data log-likelihood. On the other hand, if $z_n$ captures valuable information about $x_n$, then the decoder can produce a distribution that is tailored specifically to $x_n$ and can thus achieve a relatively high observed data log-likelihood across the $x_n$ in the training set. 

However, because the weights in the encoder and decoder are initialized randomly, there is no significant relationship between $z$ and $x$ in the encoder at the start of training. Instead, $\mathbf{z}$ is nearly independent of $\mathbf{x}$ because for a given value of $x$, the encoder is essentially outputting a random value for $z$ that doesn't capture the latent representation of $x$. Therefore, once training starts, the approximate posterior $q_{\phi}(\mathbf{z}|\mathbf{x})$ does't deviate substantially from its prior of $N(0, I)$. This is the case because given the penalty incurred for diverging from the prior, the encoder is only incentivized to output latent variables that have low likelihood under the prior if doing so would transmit information to the decoder that would significantly help the deocder reconstruct the original data. However, because $x$ and $z$ are initially independent in the encoder and the decoder, there is no reason for the encoder to stray from the prior.  

Meanwhile, in order to increase the marginal data log likelihood term, gradient descent causes the decoder to learn weights that approximate $x$ without using $z$. As this learning process continues, $p_{\theta}(x|z) \approx p_{\theta}(x)$ changes. This, in turn, means that the true posterior $p_{\theta}(\mathbf{z}|\mathbf{x})$, which is proportional to $p(z)p_{\theta}(x|z)$ becomes increasingly different from the approximate prior, which, as explained above, remains anchored at the prior. As a result, the KL divergence term increases in magnitude and has a larger influence on the gradient. Eventually, the decoder learns weights that come close to maximizing the observed data log likelihood subject to the limitation that $z$ conveys no useful information about $x$. At this point, the gradient associated with the KL divergence term dominates and pushes the true posterior and approximate posterior into alignment. To achieve this, the gradient simply drives the true posterior to the prior. Because the decoder doesn't make use of $z$ in the first place, this doesn't impede the decoder's ability produce an average representation of the training data. 

At this point, the VAE is in a state of posterior collapse as the model still hasn't learned a useful latent encoding. And unfortunately, this state is locally optimal because both terms in the ELBO have been locally optimized -- from the perspective of the marginal data log likelihood term, the VAE is doing the best it can at reconstructing $x$, while from the persepctive of the KL divergence term, the approximate and true posteriors completely overlap so there's no room for improvement. Therefore, performing additional rounds of gradient descent don't help to escape from this state of posterior collapse.

The authors validate this reasoning by observing the behavior of the encoder and decoder over the course of training for a simple VAE model. This toy example consists of an LSTM encoder and decoder and a one-dimensional latent variable, such that it's easy to visualize the mean of the true posterior and approximate posterior at different stages of the training process. The results are displayed below:  

![](figs/collapse.png)

In the figure above, the x-axis represents the mean of the true posterior ($p_{\theta}(z|x) = \frac{p_{\theta}(x|z)p(z)}{p_{\theta}(x)}$), while the y-axis represents the mean of the approximate posterior ($q_{\phi}(z|x)$). Each "X" corresponds to different training point. Initially, both of these means are centered at the origin, as shown in the left-most plot. The reason for this is that the weights of the encoder and decoder LSTMs are randomly initialized from a uniform distribution with a mean of zero, so by symmetry, the expected output of the encoder and decoder is zero. The second plot shows that when training starts, the points spread out across the x-axis, which indicates that the true posterior does not suffer from posterior collapse. This is consistent with the discussion above about how the observed data log likelihood term pushes the decoder to capture the average of the training examples. On the other hand, none of the points in the second plot spread out along the vertical axis, which indicates that the encoder is in a state of inference collapse and that the approximate posterior doesn't digress from the prior. Finally, the third and fourth plots show that as training proceeds the true posterior gradually shrinks back toward the prior such that the true posterior and approximate posterior are in agreement with each other. 

## Technical content (details)

The discussion above reveals that posterior collapse is caused by the failure of the encoder to produce a meaningful latent representation or, in other words, to approximate the true posterior. This suggests that standard VAE training, which involves simultaneously differentiating the ELBO with respect to the weights of the encoder and decoder and simultatenously updating the weights of both networks, causes the encoder to lag far behind the true posterior, which is a moving target that changes during every iteration. This realization, in turn, motivates the introduction of a very simple modification to the standard VAE training algorithm: rather than training the encoder and decoder simultaneously, we should update the two networks separately. First, we should train the encoder for as long as it takes to catch up to the true posterior and only then should we proceed to update the decoder, which will cause the true posterior to shift once more.

To be concrete, standard VAE training involves repeating the following three steps until convergence:
1. Sample a random data minibatch from the training set
2. Use automatic differentiation to compute the gradient of the elbo with respect to the weights of the encoder and decoder.
3. Update the weights of the encoder and decoder by taking a step in the direction opposite of the gradient. 

The proposed modification, called aggressive training, converts this procedure into a two stage, coordinate ascent-style algorithm:

1. Force the approximate posterior to catch up to true posterior by repeating the following three steps until the encoder converges:
    1. Sample a random data minibatch from the training set
    2. Compute the gradient of the elbo with respect to the weights of the encoder
    3. Update the weights of the encoder by taking a step in the direction opposite of the gradient
2. Train the decoder by repeating the following three steps once:
    1. Sample a random data minibatch from the training set
    2. Compute the gradient of the elbo with respect to the weights of the decoder
    3. Update the weights of the decoder by taking a step in the direction opposite of the gradient

The diagram below illustrates what's taking place during the aggressive training:

![](figs/aggressive.png)

Each horizontal shift reflects the change in the true posterior that occurs each time the decoder is updated (i.e. step 2 of the algorithm). At this point, the approximate posterior outputted by the encoder no longer matches the true posterior. Therefore, the next step is to train the encoder network for as long as it takes to capture the new true posterior (i.e. step 1 of the algorithm). This update to the approximate posterior is reflected by the vertical shift, which pushes the points back toward the diagonal line such that the true posterior and approximate posterior are in alignment. 

While the modified training algorithm presented above ensures that the approximate posterior keeps up with the true posterior, it is significantly more computational intensive that the original algorithm because it involves updating the encoder weights an arbitrary number of times during each iteration. Therefore, we only want to carry out the modified algorithm when we're concerned that the model will fall into a state of posterior collapse. On the other hand, if we're confident that the model will avoid posterior collapse, then we should revert back to the original algorithm where the encoder and encoder are trained together. The final step, therefore, is to devise a way to assess the risk of suffering from posterior collapse at an arbitrary stage of the training process. 

The fact that posterior collapse occurs when the encoder fails to learn a meaningful latent representation of the training data suggests that we develop a metric that quantifies the degree to which the encoder is achieving it's goal. The authors propose using the mutual information $I_q$ between $x$ and $z$ under the approximate posterior $q_{\phi}(\mathbf{z}|\mathbf{x})$. In information theory, the mutual information between two random variables measures the dependencve between the random variable or, in other words, the information about one random variable provided by the other random variable. Therefore, the higher the mutual information between x and z, the more the encoder is learning a latent representation that is tailored to each training example, and the more confident we are that the approximate posterior is no longer concentrated at the prior. On the other hand, a low mutual information between x and z indicates that x and z are still nearly independent, which, as explained above, means that the approximate posterior is lagging far behind the true posterior. 

This mutual information metric is easily incorporated into the modified training algorithm: after each cycle that alternates between training the encoder and training the decoder, the mutual information is calculated on a held-out validation set. When the mutual information stops increasing, the algorithm reverts back to standard VAE training, which continues until the convergence of the entire VAE.

The paper presents the following formulation of the mutual information between $x$ and $z$ under $q_{\phi}(\mathbf{z}|\mathbf{x})$:

$$ I_q = E_{\mathbf{x} \sim p_d(\mathbf{x})}[D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x})||p(\mathbf{z})] - D_{KL}(q_{\phi}(\mathbf{z})||p(\mathbf{z})$$

where $\mathbf{x} \sim p_d(\mathbf{x})$ denotes the empirical distribution defined by the training set. However, in the authors' implementation of their methodology, they compute the following alternative form of the mutual information:

$$ I_q = E_{\mathbf{x} \sim p_d(\mathbf{x})}[E(q_{\phi}(\mathbf{z}|\mathbf{x})log(q_{\phi}(\mathbf{z}|\mathbf{x}))] - E_{\mathbf{x} \sim p_d(\mathbf{x})}[E(q_{\phi}(\mathbf{z}|\mathbf{x})log(q_{\phi}(\mathbf{z}))]$$



Finally, the authors verify the effectiveness of agressive training by using their proposed training procedure on the toy VAE model with one latent variable. The results are displayed below:

![](figs/diagonal.png)

As before, the first plot shows that the means of both the true posterior and approximate posterior are located at the origin upon initialization. The rest of the plots show that all the points lie roughly along the diagonal, which indicates that the encoder is outputting an approximate posterior than is very close to the true posterior. Overall, this shows that aggressive training has successfully avoided posterior collapsed. 